In [1]:
import stella_lstm

In [2]:
import os, sys, inspect

current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0,parent_dir)

import numpy as np
from numpy import newaxis
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers.recurrent import LSTM
from keras.models import Sequential
import time #helper libraries

import matplotlib.pyplot as plt

c:\users\tuan\appdata\local\conda\conda\envs\deep\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
X_train, y_train, X_test, y_test = stella_lstm.load_data('../data/BCCUSDT.csv', sequence_length=50, future_shift=20)

      event_time ignore     trade_time  seller_order_id  buyer_order_id  \
0  1518213695334   True  1518213695332          8113161         8113114   
1  1518213698228   True  1518213698225          8113161         8113163   
2  1518213699673   True  1518213699675          8113161         8113164   
3  1518213729324   True  1518213729321          8113192         8113164   
4  1518213729324   True  1518213729321          8113192         8113165   

  event_type is_buyer_market_maker    price  quantity    symbol  trade_id  
0      trade                 False  1278.84   0.03879   BCCUSDT   1360256  
1      trade                  True  1278.84   0.19120   BCCUSDT   1360257  
2      trade                  True  1278.84   0.92646   BCCUSDT   1360258  
3      trade                 False  1278.84   0.00154   BCCUSDT   1360259  
4      trade                 False  1278.84   0.02794   BCCUSDT   1360260  


In [10]:
print(X_train[0])

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00 -3.17475212e-03 -3.45625723e-03
  -2.80723155e-03 -3.09655625e-03 -3.09655625e-03  0.00000000e+00
  -1.73594820e-03 -1.73594820e-03 -3.60482938e-03 -2.92452535e-03
  -2.92452535e-03 -2.92452535e-03 -2.92452535e-03  0.00000000e+00
   0.00000000e+00  0.00000000e+00 -8.83613275e-04 -2.82287073e-03
  -2.82287073e-03 -1.55609771e-03 -2.82287073e-03 -2.82287073e-03
  -1.68121110e-03 -2.82287073e-03 -2.92452535e-03 -2.96362328e-03
  -3.19821088e-03 -3.19821088e-03 -4.17565919e-03 -3.29986550e-03
  -3.29986550e-03 -3.51881393e-03 -3.51881393e-03 -3.51881393e-03
  -3.51881393e-03 -3.29986550e-03 -3.29986550e-03 -5.05145288e-03
  -5.05927247e-03 -5.06709205e-03 -5.08273123e-03 -5.05145288e-03
  -5.05145288e-03 -5.05145288e-03]
 [ 0.00000000e+00  1.90552885e-09  2.86059865e-09  2.23874941e-08
   2.23874941e-08  3.51044127e-08  3.73168811e-08  3.73168811e-08
   4.76164852e-08  4.76164852e-08  5.0517

In [7]:
print(y_test[0])

-0.6585000000000264


In [ ]:
#Step 2 Build Model
model = Sequential()

model.add(LSTM(
    input_shape=(X_train.shape[1],X_train.shape[2]),
    units=50,
    return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(
    100,
    return_sequences=True))
model.add(Dropout(0.2))

model.add(Flatten()) # added for dimensionality reduction

model.add(Dense(
    units=1))
model.add(Activation('linear'))

start = time.time()
model.compile(loss='mse', optimizer='rmsprop')
print('compilation time : ', time.time() - start)

In [ ]:
#Step 3 Train the model
model.fit(
    X_train,
    y_train,
    batch_size=512,
    epochs=5,
    validation_split=0.05)

In [ ]:
prediction = model.predict(X_test)

In [ ]:
plt.plot(prediction[:500], color='b')
plt.plot(y_test[:500], color='r')
plt.show()

In [ ]:
threshold = 0.02

success = 0
fail = 0

for i in range(len(prediction)):
    if (prediction[i] > threshold) and (y_test[i] > 0):
        success += 1
    elif (prediction[i] < -threshold) and (y_test[i] < 0):
        success += 1
    elif (prediction[i] < threshold)and (prediction[i] > -threshold):
        pass
    else:
        fail += 1
float(success / (success + fail))

In [ ]:
success = 0
fail = 0

for i in range(len(prediction)):
    if (prediction[i] > 0) and (y_test[i] > 0):
        success += 1
    elif (prediction[i] < 0) and (y_test[i] < 0):
        success += 1
    else:
        fail += 1
float(success / (success + fail))

In [ ]:
model.save('stella_lstm_BCCUSDT.h5')